In [ ]:
import gensim
import numpy as np
import pandas as pd



In [ ]:
from sklearn.model_selection import train_test_split
pd.set_option('display.max_colwidth', 100)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


In [ ]:
! ls

In [ ]:
%cd gdrive/My Drive/Colab Notebooks/Webpages_Classification_test_data.csv

In [ ]:
messages = pd.read_csv('Webpages_Classification_test_data.csv')
messages.head()

In [ ]:
messages = messages.drop(labels = ["url","url_len","ip_add","geo_loc","tld","who_is","https","js_len","js_obf_len"], axis = 1)
messages.head()


In [ ]:
messages=messages.iloc[:10000]

In [ ]:
messages.count()

In [ ]:
messages = messages.iloc[: , 1:]

In [ ]:
messages.head()

In [ ]:
messages['content_clean'] = messages['content'].apply(lambda x: gensim.utils.simple_preprocess(x))
messages.head()

In [ ]:
messages['label']=messages['label'].map({'good':1,'bad':0})

In [ ]:
X_train, X_test, y_train, y_test = train_test_split (messages['content_clean'], messages['label'] , test_size=0.2)

In [ ]:
w2v_model = gensim.models.Word2Vec(X_train,
                                   vector_size=100,
                                   window=5,
                                   min_count=2)

In [ ]:
w2v_model.wv.index_to_key

In [ ]:
print(gensim.__version__)

In [ ]:
!pip install gensim

In [ ]:
print(gensim.__version__)

In [ ]:
!pip install --upgrade gensim

In [ ]:
words = set(w2v_model.wv.index_to_key )
X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in X_train])
X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in X_test])

In [ ]:
for i, v in enumerate(X_train_vect):
    print(len(X_train.iloc[i]), len(v))

In [ ]:
X_train_vect_avg = []
for v in X_train_vect:
    if v.size:
        X_train_vect_avg.append(v.mean(axis=0))
    else:
        X_train_vect_avg.append(np.zeros(75, dtype=float))
        
X_test_vect_avg = []
for v in X_test_vect:
    if v.size:
        X_test_vect_avg.append(v.mean(axis=0))
    else:
        X_test_vect_avg.append(np.zeros(75, dtype=float))

In [ ]:
for i, v in enumerate(X_train_vect_avg):
    print(len(X_train.iloc[i]), len(v))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf_model = rf.fit(X_train_vect_avg, y_train.values.ravel())

In [ ]:
y_pred = rf_model.predict(X_test_vect_avg)

In [ ]:
from sklearn.metrics import precision_score, recall_score
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print('Precision: {} / Recall: {} / Accuracy: {}'.format(
    round(precision, 3), round(recall, 3), round((y_pred==y_test).sum()/len(y_pred), 3)))